In [19]:
import pandas as pd 
import numpy as np
import os 
import glob 
import ast
import nest_asyncio         # nest_asyncio는 파이썬의 비동기 코드 실행을 도와주는 패키지
nest_asyncio.apply()        # Jupyter 노트북과 같이 이벤트 루프가 이미 실행 중인 환경에서 비동기 코드를 여러 번 실행할 수 있도록 설정하는 데 사용
# 이거 안하면 무조건 에러남 

from datasets import Dataset 
from ragas import evaluate

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,      # component-wise 평가 방법
    answer_correctness,
    answer_similarity       # end-to-end 평가 방법
    
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
]


In [22]:
def set_data(file):
    qa_data = pd.read_excel(file)
    for col in ['contexts']:
        qa_data[col] = qa_data[col].apply(ast.literal_eval)
        
    dataset = Dataset.from_pandas(qa_data)
    
    return qa_data, dataset

In [23]:
file = 'experiment_dataset/QA/ablation_study_best_retrievers_001.xlsx'

qa_data, dataset = set_data(file)

In [24]:
qa_data

,question,ground_truth,answer,contexts
0,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 몇 건인가요?,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 3건입니다.,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 3건입니다.,[13\n3 반덤핑 제도 운영상의 주요 특징\n인 도에서 가장 많이 활용해 온 반덤...
1,A사는 인도에서 어떤 제품을 생산하고 있나요?,A사는 인도에서 스마트폰을 생산하고 있습니다.,A사가 인도에서 어떤 제품을 생산하는지에 대한 정보는 제공된 자료에 명시되어 있지 ...,[인도 전문 식품수입 벤더현황\n1. 요 약 문\n가. 인도 농식품 유통현황 및 시...
2,인도의 국경 간 무역 용이성 수준은 얼마인가요?,인도의 국경 간 무역 용이성 수준은 77.46입니다.,인도의 국경 간 무역 용이성 수준은 77.46입니다. (자료: World Bank ...,[11\n입\n도\n|\n절Ⅰ제\n인도 통관·통상환경\n국경 간 무역 용이성 수준\...
3,한-인도 CEPA 적용을 위한 원산지증명서 서식은 무엇에 따라 달라질 수 있나요?,한-인도 CEPA 적용을 위한 원산지증명서 서식은 한국과 인도의 양국에서 사용하는 ...,한-인도 CEPA 적용을 위한 원산지증명서 서식은 한-인도 CEPA 협정에 따라 정...,[2. 원산지증명서\n가. 원산지증명서 오류 발급 시 정정 방법\n① 주요 내용\n...
4,인도의 무역구제규정 개정의 주요 내용은 무엇인가요?,"인도의 무역구제규정 개정의 주요 내용은 WTO법과의 합치성을 제고하고, 특정 용어에...",인도의 무역구제규정 개정의 주요 내용은 다음과 같습니다:\n- 2020년 2월 2일...,"[제Ⅳ절\n인도 통상정책\n02\n인도의\n무역구제조치\n, 제Ⅳ절\n인도 통상정책..."
5,BIS 인증을 위해 샘플을 인도에 발송할 때 주의해야 할 사항은 무엇인가요?,BIS 인증을 위해 샘플을 인도에 발송할 때는 글로벌 특송업체를 이용하는 것이 좋으...,BIS 인증을 위해 샘플을 인도에 발송할 때 주의해야 할 사항은 다음과 같습니다:\...,[<BIS 지정연구소 리스트 확인 화면>\n자료: Bureau of Indian S...
6,인도에 중고 설비를 수출할 때 한-인도 CEPA를 활용하기 위해 필요한 서류는 무엇...,중고 설비를 수출할 때 한-인도 CEPA를 활용하기 위해서는 수출국의 공인 감정평가...,인도에 중고 설비를 수출할 때 한-인도 CEPA를 활용하기 위해 필요한 주요 서류는...,[제Ⅱ절\n한-인도 CEPA 활용방안\n02\n한-인도 CEPA\n수출유망품목 및\...
7,인도 무역구제제도 운영의 최근 변화는 무엇인가요?,최근 인도 무역구제제도 운영에서는 법규 제·개정 및 매뉴얼 작업을 통해 조사 과정상...,인도의 무역구제제도 운영의 최근 변화는 다음과 같습니다:\n- 2020년 2월 12...,"[제Ⅳ절\n인도 통상정책\n01\n최근 인도의\n통상정책 동향\n, 제Ⅳ절\n인도 ..."
8,인도에서 식품 수입 시 필요한 필수 서류는 무엇인가요?,인도에서 식품 수입 시 필요한 필수 서류는 인도 상공부 무역국(DGFT)에서 발행한...,인도에서 식품을 수입할 때 필요한 필수 서류는 다음과 같습니다:\n1. 수출입코드와...,[인도 전문 식품수입 벤더현황\n▪\n서류검토: 인도 식품 당국에 의해 서류 검토가...
9,FSSAI 라이선스 발급 절차는 몇 단계로 이루어져 있나요?,FSSAI 라이선스 발급 절차는 3단계로 이루어져 있습니다.,FSSAI 라이선스 발급 절차는 총 3단계로 이루어져 있습니다.\n\n1. 1단계:...,[2019년 해외 이슈조사 보고서\n6. FSSAI 라이선스 발급제도\n가. FSS...


In [7]:
qa_data['contexts'][0]

['13\n3 반덤핑 제도 운영상의 주요 특징\n인 도에서 가장 많이 활용해 온 반덤핑 제도를 중심으로 인도의 조사절차 및 마진산정 상의 주요\n특징을 살펴봄\n○ 분 석에는 ①2018년 1월1일 이후 개시되고 2020년 9월 1일까지 최종판정이 나온 반덤핑 조사 판정문\n분석과 ②인도 무역구제를 오랫동안 대응해온 업체 및 협회, 국내외 전문가의 인터뷰를 주로 활용\n<인도 반덤핑 제도 운영상의 주요 특징과 유의사항>\n구분 주요 특징과 유의사항\n(1) 조사 절차 개관 – 예비판정 생략, 절차적 요건 준수 경향 강화\n조사 절차에서 발견된 특징\n(2) 규제 존속 기간 – 잠정조치 발동시 잠정조치일로부터 5년 부과\n(3) 비관계상사의 답변 의무 (밸류체인의 완전성)\n(4) 불리한 이용가능정보(AFA) 적용\n마진 산정 및 관세 부과\n(5) 인도 국내산업의 비생산 모델에 대한 반덤핑 조치\n과정에서 발견된 특징\n(6) 높은 수준의 조사(재심) ‘개시’대비 실제 ‘조치’비율\n(7) 반덤핑/상계관세 부과 방식과 인도의 총 관세 산정 방식\n(1) 조사 절차 개관\n( 조사기간) 2018년 1월1일 이후 개시되고 최종판정이 나온 케이스들을 집계하여 살펴본 결과,\n반덤핑 원심 조사의 평균 조사기간은 약 317.8일로 미국의 원심 평균 조사 기간인 약 3299)일과\n유사한 수준임\n○ W TO 반덤핑 협정상 조사는 일반적으로 1년 이내에 종결된다고 정하는데, 인도 상무부 반덤핑위원회는\n내부적으로 조사개시일로부터 11개월 이내 조사 완료 지침을 마련함10)\n○ 일 몰재심과 우회덤핑조사의 평균조사기간은 각각 233.5일, 235일로서 원심조사에 비해 신속하게 진행\n되는 편이며, 중간 재심이나 신규수출자재심은 개시에서 판정까지 약 1년 이상의 장기 시간이 소요됨\n9) 2 008년 1월1일 이후 개시되고 2019년 7월31일까지 최종판정이 나온 원심조사 263건의 평균 조사 기간임.\n10) 업 계·전문가 인터뷰 참고. 원심의 조사기간은 WTO 반덤핑 협정

In [8]:
len(qa_data)

11

In [9]:
# from ragas import evaluate

# metrics_gen = [faithfulness,answer_relevancy]

# result = evaluate(
#     dataset,
#     metrics=metrics_gen,
#     raise_exceptions=False  # Add this line
# )

In [10]:
# df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'setting1'])


# df1

In [11]:
# df1.set_index('Metric', inplace=True)
# df1 = df1.transpose().reset_index()

# df1

In [25]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting4*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

In [26]:
file_list[:]

['./experiment_dataset/QA\\setting4_mq_retriever_001.xlsx',
 './experiment_dataset/QA\\setting4_mq_retriever_002.xlsx',
 './experiment_dataset/QA\\setting4_mq_retriever_003.xlsx']

In [27]:
len(file_list)

3

In [28]:
file_list[0].split('\\')[-1].split('.')[0]

'setting4_mq_retriever_001'

In [29]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings
from ragas.embeddings import HuggingfaceEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

from ragas.run_config import RunConfig
# RAGAS Evaluation TimeoutError를 해결하기 위해 

# llm = ChatOllama(model="gemma2")
# llm = Ollama(model="llama3")
llm = ChatOpenAI(model= 'gpt-4o')

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")    
# embeddings = OllamaEmbeddings(model="gemma2")
# embeddings = OllamaEmbeddings(model="llama3")


# embeddings = FastEmbedEmbeddings(model_name="jhgan/ko-sroberta-multitask")
# embeddings = HuggingfaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
# embeddings = HuggingfaceEmbeddings(model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr")

In [30]:
def merge_result(file_list):
    for i, file in enumerate(file_list): 
        nest_asyncio.apply() 
        set_name = file.split('\\')[-1].split('.')[0]
        print(set_name)
        _, dataset = set_data(file)
        # raise_exceptions=False로 하면 평가시 nan값이 발생했을 때, 평가 종합 계산시 제외한다.
        result = evaluate(dataset, 
                          metrics=metrics, 
                          raise_exceptions=False, 
                          run_config=RunConfig(timeout=480, max_retries=10, max_wait=480, max_workers=16),
                          llm = llm,
                          embeddings=embeddings
                          )
        
        if i == 0:
            df = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])

        else:
            df_temp = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])
            df = pd.merge(df, df_temp, on='Metric')
        
        print(df)
    
    
    df.set_index('Metric', inplace=True)
    df = df.transpose().reset_index()
    
    return df
           

In [31]:
import nest_asyncio     
nest_asyncio.apply()  

In [32]:
df = merge_result(file_list)

setting4_mq_retriever_001


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_mq_retriever_001
0        faithfulness                   0.855736
1    answer_relevancy                   0.723876
2      context_recall                   0.818182
3   context_precision                   0.562121
4  answer_correctness                   0.584669
5   answer_similarity                   0.944629
6                mean                   0.748202
setting4_mq_retriever_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_mq_retriever_001  setting4_mq_retriever_002
0        faithfulness                   0.855736                   0.890152
1    answer_relevancy                   0.723876                   0.820160
2      context_recall                   0.818182                   0.727273
3   context_precision                   0.562121                   0.613636
4  answer_correctness                   0.584669                   0.585622
5   answer_similarity                   0.944629                   0.954260
6                mean                   0.748202                   0.765184
setting4_mq_retriever_003


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_mq_retriever_001  setting4_mq_retriever_002  \
0        faithfulness                   0.855736                   0.890152   
1    answer_relevancy                   0.723876                   0.820160   
2      context_recall                   0.818182                   0.727273   
3   context_precision                   0.562121                   0.613636   
4  answer_correctness                   0.584669                   0.585622   
5   answer_similarity                   0.944629                   0.954260   
6                mean                   0.748202                   0.765184   

   setting4_mq_retriever_003  
0                   0.859668  
1                   0.732147  
2                   0.818182  
3                   0.578788  
4                   0.509955  
5                   0.943046  
6                   0.740298  


In [33]:
df

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting4_mq_retriever_001,0.855736,0.723876,0.818182,0.562121,0.584669,0.944629,0.748202
1,setting4_mq_retriever_002,0.890152,0.820160,0.727273,0.613636,0.585622,0.954260,0.765184
2,setting4_mq_retriever_003,0.859668,0.732147,0.818182,0.578788,0.509955,0.943046,0.740298


In [24]:
df.to_excel('./experiment_dataset/ablation_study/best_retriever3_emb_gemma2.xlsx', index=False)

In [12]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting2_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

file_list

['./experiment_dataset/QA\\setting2_retriever2_i003_j003_m005.xlsx',
 './experiment_dataset/QA\\setting2_retriever2_i005_j003_m005.xlsx',
 './experiment_dataset/QA\\setting2_retriever2_i007_j003_m005.xlsx',
 './experiment_dataset/QA\\setting2_retriever2_i010_j003_m005.xlsx']

In [13]:
import nest_asyncio     
nest_asyncio.apply()  

In [17]:
df = merge_result(file_list)

setting2_retriever2_i003_j003_m005


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_i003_j003_m005
0        faithfulness                            0.774242
1    answer_relevancy                            0.752225
2      context_recall                            0.636364
3   context_precision                            0.636364
4  answer_correctness                            0.534286
5   answer_similarity                            0.943716
6                mean                            0.712866
setting2_retriever2_i005_j003_m005


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_i003_j003_m005  \
0        faithfulness                            0.774242   
1    answer_relevancy                            0.752225   
2      context_recall                            0.636364   
3   context_precision                            0.636364   
4  answer_correctness                            0.534286   
5   answer_similarity                            0.943716   
6                mean                            0.712866   

   setting2_retriever2_i005_j003_m005  
0                            0.831841  
1                            0.931366  
2                            0.818182  
3                            0.719697  
4                            0.502947  
5                            0.950641  
6                            0.792446  
setting2_retriever2_i007_j003_m005


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_i003_j003_m005  \
0        faithfulness                            0.774242   
1    answer_relevancy                            0.752225   
2      context_recall                            0.636364   
3   context_precision                            0.636364   
4  answer_correctness                            0.534286   
5   answer_similarity                            0.943716   
6                mean                            0.712866   

   setting2_retriever2_i005_j003_m005  setting2_retriever2_i007_j003_m005  
0                            0.831841                            0.800699  
1                            0.931366                            0.828969  
2                            0.818182                            0.818182  
3                            0.719697                            0.727273  
4                            0.502947                            0.459226  
5                            0.950641                  

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_i003_j003_m005  \
0        faithfulness                            0.774242   
1    answer_relevancy                            0.752225   
2      context_recall                            0.636364   
3   context_precision                            0.636364   
4  answer_correctness                            0.534286   
5   answer_similarity                            0.943716   
6                mean                            0.712866   

   setting2_retriever2_i005_j003_m005  setting2_retriever2_i007_j003_m005  \
0                            0.831841                            0.800699   
1                            0.931366                            0.828969   
2                            0.818182                            0.818182   
3                            0.719697                            0.727273   
4                            0.502947                            0.459226   
5                            0.950641            

In [18]:

df

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting2_retriever2_i003_j003_m005,0.774242,0.752225,0.636364,0.636364,0.534286,0.943716,0.712866
1,setting2_retriever2_i005_j003_m005,0.831841,0.931366,0.818182,0.719697,0.502947,0.950641,0.792446
2,setting2_retriever2_i007_j003_m005,0.800699,0.828969,0.818182,0.727273,0.459226,0.941208,0.762593
3,setting2_retriever2_i010_j003_m005,0.935606,0.844105,0.818182,0.727273,0.483344,0.944656,0.792194


In [29]:
df.to_excel('./experiment_dataset/QA_results/setting2_retriever2.xlsx', index=False)

In [30]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting3_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting3_retriever3.xlsx', index=False)

['./experiment_dataset/QA\\setting3_retriever3_003_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_002.xlsx', './experiment_dataset/Q

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001
0        faithfulness                     0.750541
1    answer_relevancy                     0.771468
2      context_recall                     0.545455
3   context_precision                     0.636364
4  answer_correctness                     0.357326
5   answer_similarity                     0.927871
6                mean                     0.664837
setting3_retriever3_003_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  
0                     0.742965  
1                     0.776203  
2                     0.545455  
3                     0.636364  
4                     0.416678  
5                     0.932985  
6                     0.675108  
setting3_retriever3_003_003


...

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

In [31]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting4_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting4_our_retriever.xlsx', index=False)

['./experiment_dataset/QA\\setting4_our_retriever_003_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_007_001.xlsx', './experiment_dataset/QA\\setting4_our_

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001
0        faithfulness                        0.818182
1    answer_relevancy                        0.771211
2      context_recall                        0.636364
3   context_precision                        0.636364
4  answer_correctness                        0.509174
5   answer_similarity                        0.943704
6                mean                        0.719166
setting4_our_retriever_003_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  
0                        0.798485  
1                        0.772628  
2                        0.636364  
3                        0.636364  
4                        0.503659  
5                        0.941787  
6                        0.714881  
setting4_our_retriever_003_003


...

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

In [40]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting5_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting5_mq_retriever.xlsx', index=False)



['./experiment_dataset/QA\\setting5_mq_retriever.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_000.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_001.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_002.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_003.xlsx']
setting5_mq_retriever


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever
0        faithfulness               0.897619
1    answer_relevancy               0.817189
2      context_recall               0.818182
3   context_precision               0.562987
4  answer_correctness               0.538979
5   answer_similarity               0.945551
6                mean               0.763418
setting5_mq_retriever_000


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000
0        faithfulness               0.897619                   0.956061
1    answer_relevancy               0.817189                   0.813061
2      context_recall               0.818182                   0.727273
3   context_precision               0.562987                   0.573485
4  answer_correctness               0.538979                   0.503129
5   answer_similarity               0.945551                   0.938030
6                mean               0.763418                   0.751840
setting5_mq_retriever_001


...

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  
0                   0.809877  
1                   0.717175  
2                   0.818182  
3                   0.580303  
4                   0.462504  
5                   0.945321  
6                   0.722227  
setting5_mq_retriever_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  setting5_mq_retriever_002  
0                   0.809877                   0.855051  
1                   0.717175                   0.831058  
2                   0.818182                   0.818182  
3                   0.580303                   0.575758  
4                   0.462504                   0.564834  
5                   0.945321                   0.94

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  setting5_mq_retriever_002  \
0                   0.809877                   0.855051   
1                   0.717175                   0.831058   
2                   0.818182                   0.818182   
3                   0.580303                   0.575758   
4                   0.462504                   0.564834   
5                   0.945321                 

In [41]:
df.head()

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting5_mq_retriever,0.897619,0.817189,0.818182,0.562987,0.538979,0.945551,0.763418
1,setting5_mq_retriever_000,0.956061,0.813061,0.727273,0.573485,0.503129,0.938030,0.751840
2,setting5_mq_retriever_001,0.809877,0.717175,0.818182,0.580303,0.462504,0.945321,0.722227
3,setting5_mq_retriever_002,0.855051,0.831058,0.818182,0.575758,0.564834,0.944891,0.764962
4,setting5_mq_retriever_003,0.878896,0.827737,0.818182,0.558442,0.557697,0.947701,0.764776


In [37]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting6_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting6_sq_retriever.xlsx', index=False)


['./experiment_dataset/QA\\setting6_sq_retriever.xlsx']
setting6_sq_retriever


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

               Metric  setting6_sq_retriever
0        faithfulness               0.258532
1    answer_relevancy               0.553217
2      context_recall               0.100000
3   context_precision               0.225000
4  answer_correctness               0.347728
5   answer_similarity               0.907992
6                mean               0.398745


In [39]:

df.head()

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting6_sq_retriever,0.258532,0.553217,0.1,0.225,0.347728,0.907992,0.398745
